In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import random
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
import joblib

**LOADING AND PREPROCESSING DATA**

In [12]:
train_df = pd.read_csv("../data/jobfair_train.csv")

train_df["global_competition_level"].fillna(0, inplace=True)

train_df.sort_values(by="league_id", inplace=True)

train_df.reset_index(inplace=True, drop=True)

columns_to_drop = ["season", "club_id", "league_id", "registration_country", "registration_platform_specific"]
train_df.drop(columns_to_drop, axis=1, inplace=True)

encoding_map = {
    '0) NonPayer': 0,
    '1) ExPayer': 1,
    '2) Minnow': 2,
    '3) Dolphin': 3,
    '4) Whale': 4
}
train_df["dynamic_payment_segment"] = train_df["dynamic_payment_segment"].map(encoding_map)

train_df.head()

,dynamic_payment_segment,cohort_season,avg_age_top_11_players,avg_stars_top_11_players,avg_stars_top_14_players,avg_training_factor_top_11_players,days_active_last_28_days,league_match_watched_count_last_28_days,session_count_last_28_days,playtime_last_28_days,league_match_won_count_last_28_days,training_count_last_28_days,global_competition_level,tokens_spent_last_28_days,tokens_stash,rests_stash,morale_boosters_stash,league_rank
0,0,1,21,3.936582,3.678486,0.530063,3,0,14,9495533,11,24,3.0,65,21,17,59,4
1,0,1,23,3.920945,3.658000,0.552545,12,9,26,19272719,9,24,0.0,91,3,16,7,5
2,0,1,23,3.140812,3.002486,0.608834,2,0,5,1529167,0,3,0.0,9,47,42,22,12
3,0,1,23,4.512800,4.206105,0.464074,10,3,75,29267781,10,43,2.0,106,7,0,67,2
4,0,2,25,3.800861,3.584876,0.633002,0,0,0,0,6,0,0.0,0,54,26,66,8


In [13]:
np.random.seed(42)

num_chunks = len(train_df) // 14

league_order = np.arange(num_chunks)

np.random.shuffle(league_order)

shuffled_df = pd.concat([train_df.iloc[x * 14: (x + 1) * 14] for x in league_order])

shuffled_df.reset_index(drop=True, inplace=True)

train_percentage = 0.8
num_train_rows = int(len(shuffled_df) * train_percentage)

num_train_chunks = num_train_rows // 14 * 14

train_df = shuffled_df.iloc[:num_train_chunks]
test_df = shuffled_df.iloc[num_train_chunks:]

X_train = train_df.drop("league_rank", axis=1)
y_train = train_df["league_rank"]

X_valid = test_df.drop("league_rank", axis=1)
y_valid = test_df["league_rank"]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

In [14]:
X_train_scaled.shape, X_valid_scaled.shape

((44240, 17), (11074, 17))

**HYPERPARAMETER OPTIMIZATION**
- Xgboost Regressor
- Random Forest Regressor

In [17]:
param_grid = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.001, 0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 2, 3],
}

xgbr_model = XGBRegressor()

grid_search = GridSearchCV(
    estimator=xgbr_model,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',
    cv=5,  
    verbose=1,
    n_jobs=-1,
)

grid_search.fit(X_train_scaled, y_train)

best_params = grid_search.best_params_
best_xgbr_model = grid_search.best_estimator_

y_pred_xgbr_best = best_xgbr_model.predict(X_valid_scaled)

mse = mean_absolute_error(y_valid, y_pred_xgbr_best)
print(f'Best Model MAE: {mse}')
print(f'Best Parameters: {best_params}')

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Model Mean Squared Error: 2.372486833385425
Best Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 300}


In [23]:
# TRAINING TIME IS TO LONG
#
#
# param_grid = {
#     'n_estimators': [100, 300, 500],
#     'max_depth': [3, 5, 7, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 3],
# }

# rfr_model = RandomForestRegressor()

# grid_search = GridSearchCV(
#     estimator=rfr_model,
#     param_grid=param_grid,
#     scoring='neg_mean_absolute_error',
#     cv=5,
#     verbose=1,
#     n_jobs=-1,
# )

# grid_search.fit(X_train_scaled, y_train)

# best_params = grid_search.best_params_
# best_rfr_model = grid_search.best_estimator_

# y_pred_rfr_best = best_rfr_model.predict(X_valid_scaled)

# mse = mean_absolute_error(y_valid, y_pred_rfr_best)
# print(f'Best Model MAE: {mse}')
# print(f'Best Parameters: {best_params}')
